In [ ]:
from haystack.dataclasses import ChatMessage
from haystack.components.generators.chat import OpenAIChatGenerator

# 建立系統消息和用戶消息的 ChatMessage 對象
# 系統消息提示生成的回應應該始終使用繁體中文，即使輸入是其他語言
# 用戶消息則是提問 "什麼是自然語言處理？要簡潔。"。
messages = [
    ChatMessage.from_system(
        "即使某些輸入資料採用其他語言，也始終以繁體中文回應。"
    ),
    ChatMessage.from_user(
        "什麼是自然語言處理？要簡潔。"
    ),
]

# 初始化 OpenAIChatGenerator
chat_generator = OpenAIChatGenerator(model="gpt-4-turbo")
# 傳入消息並運行生成對話
response = chat_generator.run(messages=messages)
# 輸出查看
print(response)

In [ ]:
from haystack.dataclasses import ChatMessage
from haystack.components.generators.chat import OpenAIChatGenerator
from haystack.components.generators.utils import print_streaming_chunk

# 使用流式回調函數初始化 OpenAIChatGenerator
chat_generator = OpenAIChatGenerator(
    model="gpt-4-turbo",
    streaming_callback=print_streaming_chunk
)
# 傳入消息並運行
response = chat_generator.run(messages=messages)

保存 OpenAI API Key 為環境變數

In [ ]:
from getpass import getpass
import os
from dotenv import load_dotenv

# 載入環境變數
load_dotenv()
# 兩個 API 的密鑰
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass("Enter OpenAI API key:")

In [ ]:
from haystack import Pipeline, Document
from haystack.document_stores.in_memory import InMemoryDocumentStore
from haystack.components.writers import DocumentWriter
from haystack.components.embedders import SentenceTransformersDocumentEmbedder

In [ ]:
# 建立文件
documents = [
    Document(content="My name is Jean and I live in Paris."),
    Document(content="My name is Mark and I live in Berlin."),
    Document(content="My name is Giorgio and I live in Rome."),
    Document(content="My name is Marta and I live in Madrid."),
    Document(content="My name is Harry and I live in London."),
]

In [ ]:
# 建立索引管道
indexing_pipeline = Pipeline()

In [ ]:
# 初始化內存文件儲存組件
document_store = InMemoryDocumentStore()    
# 在管道中加入組件：將文件內容轉換成嵌入向量
indexing_pipeline.add_component(
    instance=SentenceTransformersDocumentEmbedder(
        model="sentence-transformers/all-MiniLM-L6-v2"
    ),
    name="doc_embedder"
)
# 加入組件：將處理後的文件數據寫入到指定的文件儲存
# 指定使用 `內存文件儲存`
indexing_pipeline.add_component(
    instance=DocumentWriter(document_store=document_store),
    name="doc_writer"
)

In [ ]:
# 連接嵌入器和文件寫入器
indexing_pipeline.connect(
    "doc_embedder.documents",
    "doc_writer.documents"
)

In [ ]:
# 運行管道
indexing_pipeline.run({
    "doc_embedder": {"documents": documents}
})

In [ ]:
from utils.draw_pipeline import draw_and_display

draw_and_display(indexing_pipeline, "ex10_1_pipe.png")

In [ ]:
from haystack.components.embedders import SentenceTransformersTextEmbedder
from haystack.components.retrievers.in_memory import InMemoryEmbeddingRetriever
from haystack.components.builders import PromptBuilder
from haystack.components.generators import OpenAIGenerator

In [ ]:
# 定義提示模板
template = """
根據給定的上下文回答問題。

上下文:
{% for document in documents %}
    {{ document.content }}
{% endfor %}
問題: {{ question }}
答案:
"""

In [ ]:
# 建立 RAG 管道
rag_pipe = Pipeline()

In [ ]:
rag_pipe.add_component(
    "embedder",
    SentenceTransformersTextEmbedder(
        model="sentence-transformers/all-MiniLM-L6-v2"
    )
)
rag_pipe.add_component(
    "retriever",
    InMemoryEmbeddingRetriever(
        document_store=document_store
    )
)
rag_pipe.add_component(
    "prompt_builder",
    PromptBuilder(
        template=template
    )
)
rag_pipe.add_component(
    "llm",
    OpenAIGenerator(model="gpt-4-turbo")
)

In [ ]:
# 連接組件
rag_pipe.connect(
    "embedder.embedding",
    "retriever.query_embedding"
)
rag_pipe.connect(
    "retriever",
    "prompt_builder.documents"
)
rag_pipe.connect(
    "prompt_builder",
    "llm"
)

In [ ]:
draw_and_display(rag_pipe, "ex10_2_pipe.png")

In [ ]:
query = "Mark 住在哪裡？"
rag_pipe.run({
    "embedder": {"text": query},
    "prompt_builder": {"question": query}
})

In [ ]:
def rag_pipeline_func(query: str):
    result = rag_pipe.run({
        "embedder": {"text": query},
        "prompt_builder": {"question": query}
    })
    return {"reply": result["llm"]["replies"][0]}

In [ ]:
WEATHER_INFO = {
    "Berlin": {
        "weather": "mostly sunny", "temperature": 7, "unit": "celsius"
    },
    "Paris": {
        "weather": "mostly cloudy", "temperature": 8, "unit": "celsius"
    },
    "Rome": {
        "weather": "sunny", "temperature": 14, "unit": "celsius"
    },
    "Madrid": {
        "weather": "sunny", "temperature": 10, "unit": "celsius"
    },
    "London": {
        "weather": "cloudy", "temperature": 9, "unit": "celsius"
    },
}

def get_current_weather(location: str):
    if location in WEATHER_INFO:
        return WEATHER_INFO[location]
    else:
        # 回退數據
        return {
            "weather": "sunny",
            "temperature": 21.8,
            "unit": "fahrenheit"
        }

In [ ]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "rag_pipeline_func",
            "description": "取得有關人們居住地點的訊息",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": "搜尋中使用的查詢。從用戶的消息中推斷出這一點。它應該是一個問題或一個陳述。",
                    }
                },
                "required": ["query"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_current_weather",
            "description": "取得當前天氣",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "城市和州，例如加州舊金山"
                    }
                },
                "required": ["location"],
            },
        },
    },
]

In [ ]:
from haystack.dataclasses import ChatMessage
from haystack.components.generators.chat import OpenAIChatGenerator
from haystack.components.generators.utils import print_streaming_chunk

# 建立消息列表，包含系統消息和用戶查詢
messages = [
    ChatMessage.from_system(
        "不要假設將哪些值插入函數中。如果用戶要求不明確，請要求澄清。"
    ),
    ChatMessage.from_user("你能告訴我 Mark 住在哪裡嗎？"),
]

# 初始化 OpenAIChatGenerator
chat_generator = OpenAIChatGenerator(
    model="gpt-4-turbo",
    streaming_callback=print_streaming_chunk
)
# 傳入消息和工具列表並運行
response = chat_generator.run(
    messages=messages,
    generation_kwargs={"tools": tools}
)
# 輸出查看
print(response)

In [ ]:
import json

# 解析函數調用訊息
# 提取第一個回應中的 content
content = response['replies'][0].content

# 將 content 解析為 JSON
# content 是一個 JSON 字串，需要轉換為 Python 字典
function_calls = json.loads(content)

# 提取第一個函數調用訊息
# 提取函數調用列表中的第一個元素
function_call = function_calls[0]

# 取得函數名稱
# 取得函數名稱，這是我們需要調用的函數
function_name = function_call['function']['name']

# 解析函數參數
# 將參數解析為字典格式
function_args = json.loads(function_call['function']['arguments'])

# 輸出函數名稱和參數
print("Function Name:", function_name)
print("Function Arguments:", function_args)

# 可用函數字典
available_functions = {
    "rag_pipeline_func": rag_pipeline_func,
    "get_current_weather": get_current_weather
}

# 搜尋相應的函數並使用給定的參數調用它
if function_name in available_functions:
    # 根據函數名稱找到對應的函數
    function_to_call = available_functions[function_name]
    # 使用解包操作將參數傳遞給函數
    function_response = function_to_call(**function_args)
    # 輸出函數的返回值
    print("Function Response:", function_response)
else:
    # 如果函數名稱未找到，輸出錯誤訊息
    print(f"Function {function_name} not found.")

In [ ]:
from haystack.dataclasses import ChatMessage

# 建立函數回應消息
function_message = ChatMessage.from_function(
    content=json.dumps(function_response),
    name=function_name
)
# 將函數回應消息添加到消息列表
messages.append(function_message)

# 再次運行 OpenAIChatGenerator
response = chat_generator.run(
    messages=messages,
    generation_kwargs={"tools": tools}
)

In [ ]:
import gradio as gr
import json

from haystack.dataclasses import ChatMessage
from haystack.components.generators.chat import OpenAIChatGenerator

# 定義工具函數
def rag_pipeline_func(query: str):
    return {"reply": f"Giorgio 住在 Berlin, query was: {query}"}

def get_current_weather(location: str):
    return {"weather": "sunny", "temperature": 20, "location": location}

# 可用函數字典
available_functions = {
    "rag_pipeline_func": rag_pipeline_func,
    "get_current_weather": get_current_weather
}

chat_generator = OpenAIChatGenerator(model="gpt-3.5-turbo")
response = None
messages = [
    ChatMessage.from_system(
        "不要假設將哪些值插入函數中。"
        "如果用戶要求不明確，請要求澄清。"
    )
]

# 定義聊天機器人函數
def chatbot_with_fc(message, history):
    messages.append(ChatMessage.from_user(message))
    response = chat_generator.run(messages=messages, generation_kwargs={"tools": tools})

    while True:
        # 如果 OpenAI 回應是一個工具調用
        if response and response["replies"][0].meta["finish_reason"] == "tool_calls":
            try:
                function_calls = json.loads(response["replies"][0].content)
            except json.JSONDecodeError as e:
                print(f"解析 JSON 發生錯誤：{e}")
                break

            print(response["replies"][0])
            for function_call in function_calls:
                # 解析函數調用訊息
                function_name = function_call["function"]["name"]
                function_args = json.loads(function_call["function"]["arguments"])

                # 檢查函數是否存在
                if function_name in available_functions:
                    function_to_call = available_functions[function_name]
                    try:
                        # 使用解包操作將參數傳遞給函數
                        function_response = function_to_call(**function_args)
                    except TypeError as te:
                        print(f"函數調用錯誤：{te}")
                        continue

                    # 使用 `ChatMessage.from_function` 將函數回應添加到消息列表
                    messages.append(ChatMessage.from_function(content=json.dumps(function_response), name=function_name))
                    response = chat_generator.run(messages=messages, generation_kwargs={"tools": tools})
                else:
                    print(f"函數 {function_name} 未找到")
                    continue
        else:
            if response:
                messages.append(response["replies"][0])
            break
    return response["replies"][0].content if response else "No response generated."

# 建立聊天界面
demo = gr.ChatInterface(
    fn=chatbot_with_fc,
    examples=[
        "瑞典的首都是什麼？",
        "你能告訴我 Giorgio 住在哪裡嗎？",
        "馬德里的天氣怎麼樣？",
        "Madrid 的天氣怎麼樣？",
        "誰住在 London?",
        "Mark 住的地方的天氣怎麼樣？",
    ],
    title="請詢問有關天氣或人們居住的地方。",
)


In [ ]:
# 啟動聊天應用程序
demo.launch()